In [1]:
#Librerías
import pandas as pd
import numpy as np
import os

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html


In [2]:
#Instación de los componentes de bootstrap
#!pip install dash_bootstrap_components

This dataset contains a list of video games with sales greater than 100,000 copies. (1980-2020)

Fields include:

    Rank - Ranking of overall sales
    Name - The games name
    Platform - Platform of the games release (i.e. PC,PS4, etc.)
    Year - Year of the game's release
    Genre - Genre of the game
    Publisher - Publisher of the game
    NA_Sales - Sales in North America (in millions)
    EU_Sales - Sales in Europe (in millions)
    JP_Sales - Sales in Japan (in millions)
    Other_Sales - Sales in the rest of the world (in millions)
    Global_Sales - Total worldwide sales.

# Análisis de datos

In [3]:
df = pd.read_csv('./data/vgsales.csv',index_col = 0)
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
df.groupby('Publisher', as_index = True).agg({'Genre':'unique'})

,Genre
Publisher,
10TACLE Studios,"[Strategy, Puzzle, Adventure]"
1C Company,"[Strategy, Racing, Role-Playing]"
20th Century Fox Video Games,"[Action, Shooter]"
2D Boy,[Puzzle]
3DO,"[Action, Sports, Fighting, Shooter, Role-Playi..."
...,...
id Software,[Shooter]
imageepoch Inc.,"[Role-Playing, Adventure]"
inXile Entertainment,[Role-Playing]


In [5]:
global_sales_by_genre = df.groupby('Genre', as_index = True).agg({'Global_Sales':'sum'})

In [6]:
global_sales_by_platform = df.groupby('Platform', as_index = True).agg({'Global_Sales':'sum'})
df1 = df.groupby('Platform', as_index = True).agg({'Global_Sales':'sum'}).reset_index()

In [7]:
games_for_platform = df.groupby('Platform', as_index = True).agg({'Name':'count'})
games_for_platform.sort_values(by = ['Name'],ascending=[True])

,Name
Platform,
GG,1
PCFX,1
TG16,2
3DO,3
WS,6
SCD,6
NG,12
GEN,27
DC,52


In [8]:
global_sales_for_year = df.groupby('Year', as_index = True).agg({'Global_Sales':'sum'})
global_sales_for_year.sort_values(by = ['Global_Sales'],ascending=[True])

,Global_Sales
Year,
2017.0,0.05
2020.0,0.29
1980.0,11.38
1983.0,16.79
1987.0,21.74
1982.0,28.86
1991.0,32.23
1981.0,35.77
1986.0,37.07


In [9]:
df.groupby(
    ['Genre','Platform']
).agg(
    {
        'Global_Sales':'sum',
        'Name':'count'
    }
)

Global_Sales  Name
Genre    Platform                    
Action   2600             29.34    61
         3DS              57.02   182
         DC                1.26     3
         DS              115.56   343
         GB                7.92     6
...                         ...   ...
Strategy Wii               5.23    25
         WiiU              1.24     3
         X360             10.13    28
         XB                2.78    21
         XOne              0.38     3

[293 rows x 2 columns]

In [10]:
#
df_opt = df['Platform'].drop_duplicates()
df_opt = pd.DataFrame(df.groupby(['Publisher']).agg({'Publisher':'unique'}).apply(lambda x:x['Publisher'][0],axis =1),columns= ['Publisher'])
options_dropdown = [{'label':id_1,'value':id_2} for id_1,id_2 in zip(df_opt.index,df_opt.Publisher)]

df_sample = df[df.Platform == 'PS4']

global_sales_by_specific_platform = df_sample.groupby('Year', as_index = True).agg({'Global_Sales':'sum'})
global_sales_by_specific_platform

,Global_Sales
Year,
2013.0,24.76
2014.0,98.76
2015.0,115.30
2016.0,39.25
2017.0,0.03


In [26]:
top_videogames = df.groupby('Rank', as_index = True).agg(
    {'Name':'sum','NA_Sales':'sum','EU_Sales':'sum','JP_Sales':'sum','Other_Sales':'sum'}).iloc[:10]
top_videogames

,Name,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Rank,,,,,
1,Wii Sports,41.49,29.02,3.77,8.46
2,Super Mario Bros.,29.08,3.58,6.81,0.77
3,Mario Kart Wii,15.85,12.88,3.79,3.31
4,Wii Sports Resort,15.75,11.01,3.28,2.96
5,Pokemon Red/Pokemon Blue,11.27,8.89,10.22,1.00
6,Tetris,23.20,2.26,4.22,0.58
7,New Super Mario Bros.,11.38,9.23,6.50,2.90
8,Wii Play,14.03,9.20,2.93,2.85
9,New Super Mario Bros. Wii,14.59,7.06,4.70,2.26


In [44]:
x=top_videogames.Name
fig = go.Figure()
fig.add_trace(go.Scatter(x =x,
                        y = top_videogames.NA_Sales.values,
                        line=dict(color='peru', width=3),
                        name='NA_Sales'))
fig.add_trace(go.Scatter(x=x, y=top_videogames.EU_Sales.values,
                         line=dict(color='indigo', width=3),
                         name='EU_Sales'))
fig.add_trace(go.Scatter(x=x, y=top_videogames.JP_Sales.values,
                         line=dict(color='firebrick', width=3),
                         name='JP_Sales'))
fig.add_trace(go.Scatter(x=x, y=top_videogames.Other_Sales.values,
                         line=dict(color='lime', width=3),
                         name='Other_Sales'))

fig.show()

# Exploración de gráficas

In [13]:
#Figura 
trace = go.Scatter(x = global_sales_by_specific_platform.index,
                   y = global_sales_by_specific_platform.Global_Sales.values,
                  mode = 'lines')

fig = go.Figure([trace])

fig.update_layout(
    width = 1000,
    title = 'Titulo aleatorio',
)

fig.show()

In [14]:
trace_GS_by_platform = go.Bar(
                            x = global_sales_by_platform.index,
                            y = global_sales_by_platform.Global_Sales.values,
                        )
fig_GS_by_platform = go.Figure(data = [trace_GS_by_platform])

fig_GS_by_platform.update_layout(
    width = 1000,
    title={
        'text': "Ventas globales por plataforma",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    #title = 'Ventas globales por plataforma',
    xaxis_title="Platform",
    yaxis_title="Global Sales",
    
    xaxis = dict(
        rangeslider = dict(visible = True)
    )
)

fig_GS_by_platform.show()

In [15]:
#Figura con plotly
trace = go.Scatter(x = global_sales_by_platform.index,
                   y = global_sales_by_platform.Global_Sales.values,
                  mode = 'lines') #'markers' muestra los puntos y 'lines' las lineas -> mode = 'markers+lines'

fig = go.Figure([trace])

fig.update_layout(
    width = 1000,
    title = 'Titulo aleatorio',
)

fig.show()